In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean-test-data/clean_test_data.csv
/kaggle/input/train-clean/train_clean.csv
/kaggle/input/haikaldata/cleaned_train_v2.csv
/kaggle/input/haikaldata/cleaned_test_v2.csv
/kaggle/input/penyisihan-datavidia-7-0/sample_submission.csv
/kaggle/input/penyisihan-datavidia-7-0/train.csv
/kaggle/input/penyisihan-datavidia-7-0/test.csv


In [2]:
pip install Sastrawi

     |████████████████████████████████| 209 kB 1.3 MB/s 
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import string, re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from collections import Counter

In [4]:
df = pd.read_csv("../input/penyisihan-datavidia-7-0/train.csv")

In [5]:
df.head()

,review_id,review_text,category
0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pe...",1
1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater...",0
2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0
3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0
4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0


In [6]:
df.shape

(14856, 3)

In [7]:
def cleaning(data) :
    ##case Folding
    data = data.lower()
    
    ##remove punctuation
    remove = string.punctuation
    translator = str.maketrans(remove," "*len(remove) )
    data = data.translate(translator)
    
    ##remove stopword
    factory = StopWordRemoverFactory()
    stopwords = factory.create_stop_word_remover()
    data = stopwords.remove(data)
    
    ##stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    data  = stemmer.stem(data)
    
    return data

In [8]:
#testing
text_contoh = "Lantai kamar dan kmr mandi kotor, bau, dan.. seperti memanusiakan manusia"
print(cleaning(text_contoh))


lantai kamar kmr mandi kotor bau manusia manusia


In [9]:
#df['review_text_clean'] = df['review_text'].apply(cleaning)
#df.to_csv("")
df = pd.read_csv("../input/train-clean/train_clean.csv")

In [10]:
df['review_text_clean'] = df['review_text_clean'].astype(str)

In [11]:
token = Tokenizer()
token.fit_on_texts(df['review_text_clean'])

In [12]:
text_example = ["Aku..", "aku", "aku,","AkU"]
text_example2 = ["kurang dikurangi"]

In [13]:
vocab = len(token.index_word) + 1
vocab

10845

In [14]:
token.texts_to_sequences(text_example2)

[[2]]

In [15]:
encode = token.texts_to_sequences(df['review_text_clean'])
encode

[[30,
  284,
  5,
  262,
  339,
  12,
  24,
  27,
  438,
  54,
  68,
  108,
  274,
  5032,
  1284,
  111,
  503,
  916,
  220,
  565,
  30,
  34,
  100],
 [35, 385, 1, 62, 528, 636, 575, 1039, 29, 62, 598, 160, 1, 137, 707, 637, 42],
 [42,
  1,
  3,
  14,
  17,
  193,
  156,
  678,
  566,
  130,
  1,
  3,
  85,
  628,
  242,
  7,
  58,
  268,
  220,
  120,
  5033,
  472,
  160,
  167,
  98,
  678,
  16,
  98,
  529,
  16,
  207],
 [173, 149, 61, 5, 31, 74],
 [121],
 [5, 12, 2116],
 [417, 98, 51, 136, 83, 15, 17, 236],
 [12, 15, 16, 150, 5, 318, 71, 93, 307, 147, 439, 2947, 28],
 [1, 27, 183, 917, 47, 203, 5],
 [1, 3, 556],
 [47,
  143,
  5,
  682,
  20,
  51,
  18,
  103,
  5034,
  5035,
  5036,
  239,
  2948,
  5037,
  3639,
  27,
  566,
  3639,
  1337,
  459,
  376,
  78,
  171,
  1],
 [12, 2, 24, 585, 302, 2302, 939, 175, 1537],
 [9,
  5,
  118,
  18,
  222,
  240,
  5,
  12,
  9,
  16,
  489,
  421,
  2117,
  160,
  9,
  71,
  2949,
  120,
  93,
  5038,
  49,
  322],
 [918,
  5,
  

In [16]:
X = pad_sequences(encode, maxlen = 100, padding="post" )
y = df['category']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [18]:
X_train

array([[ 12, 162,  49, ...,   0,   0,   0],
       [ 19,  33, 176, ...,   0,   0,   0],
       [ 41,  72,   0, ...,   0,   0,   0],
       ...,
       [  2,  18,   0, ...,   0,   0,   0],
       [ 37,  88, 171, ...,   0,   0,   0],
       [ 36, 466,  35, ...,   0,   0,   0]], dtype=int32)

In [19]:
from keras.utils import to_categorical
import random
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM, SpatialDropout1D, Input
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [20]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_train, y_train)

model.score(X_test, y_test)

0.43707933918009384

In [21]:
#Using SVM
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



X_train, X_test, y_train, y_test = train_test_split(df['review_text_clean'], y, test_size=0.33, random_state=42, stratify=y)

pipeline_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', svm.SVC(kernel="linear", probability=True))
])

pipeline_tf.fit(X_train, y_train)
pipeline_tf.score(X_test,y_test)

0.9414644095451764

In [22]:
#Using Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
pos_weights = (len(y_train) - sum(y_train)) / (sum(y_train)) 
pipeline_dt = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier(max_depth=100,random_state=500, class_weight={0: 1, 1: pos_weights}))
])

pipeline_dt.fit(X_train, y_train)
pipeline_dt.score(X_test,y_test)

0.9331021823373444

In [23]:
from sklearn.neighbors import KNeighborsClassifier


pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', KNeighborsClassifier(n_neighbors=100)),
])

pipeline_nb.fit(X_train, y_train)
pipeline_nb.score(X_test,y_test)

0.9065878033856822

In [24]:
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

pipeline_nb.fit(X_train, y_train)
pipeline_nb.score(X_test,y_test)

0.881705078523353

In [25]:
test_data = pd.read_csv("../input/penyisihan-datavidia-7-0/test.csv")
test_data.head()

,review_id,review_text
0,7302180ac7160f04a405d8aa7bd6beb8,"Kasur rusak, punggung saya sakit, kasurnya ga ..."
1,3184c670e170f565c7522eb76a320ba1,gerah
2,e33abf6bb5d5a9a77c339043b1725dc8,"tempat ridur atas bawah.. ac nya pararel, tdk ..."
3,061d388950340070a6ac03fabc9027b0,"ac kurang dingin, tidak ada snack sperti di foto"
4,e62d87c348674b6c06856964f3ab16b8,Lumayan untuk guest house dengan standart harg...


In [26]:
#clean_test_data = test_data.review_text.apply(cleaning)
clean_test_data = pd.read_csv("../input/clean-test-data/clean_test_data.csv")
clean_test_data.tail()

,Unnamed: 0,review_text
3709,3709,kamar 214 kulkas dibagusin
3710,3710,buruk sekali sangat kecewa
3711,3711,lampu redup bau pengap kamar mandi ga gantung ...
3712,3712,kamar mawar bisa kunci luar suasana hotel sera...
3713,3713,overall oke lah buat singgah


In [27]:
clean_test_data['review_text'] = clean_test_data['review_text'].astype(str)
test_encode = token.texts_to_sequences(clean_test_data['review_text'])
test = pad_sequences(test_encode, maxlen = 100, padding="post" )

In [28]:
res = pipeline_dt.predict(clean_test_data['review_text'])

In [29]:
test_data['category'] = res

In [30]:
submission = test_data.drop("review_text", axis=1)
submission.to_csv("submission_dt.csv", index=False)

In [31]:
submission

,review_id,category
0,7302180ac7160f04a405d8aa7bd6beb8,0
1,3184c670e170f565c7522eb76a320ba1,0
2,e33abf6bb5d5a9a77c339043b1725dc8,0
3,061d388950340070a6ac03fabc9027b0,0
4,e62d87c348674b6c06856964f3ab16b8,1
...,...,...
3709,4fac4c272d9194021f921138ebd5b389,0
3710,4fabd4722dd4074841f9656f75105f09,0
3711,44ff89997547d54cccb2a33776181240,0
3712,c46cf84e3665a10199a539fee919edd6,0


In [32]:
df2 = pd.read_csv("../input/haikaldata/cleaned_train_v2.csv")
df2.head()

,Unnamed: 0,review_id,review_text,category,emoji_to_text,no_punctuation,lowered,no_number,no_stopwords,stemmed
0,0,0fc651bfdd38a3b4ba19ef9d0d1e92e3,"Penginapannya rapi, bersih, suasana tenang, pe...",1,"Penginapannya rapi, bersih, suasana tenang, pe...",Penginapannya rapi bersih suasana tenang pelay...,penginapannya rapi bersih suasana tenang pelay...,penginapannya rapi bersih suasana tenang pelay...,penginapannya rapi bersih suasana tenang pelay...,inap rapi bersih suasana tenang layan ramah ba...
1,1,7d8cb2e465c5602e51c1ba8710e8ff16,"wifi tidak sampai kamar, tidak tersedia heater...",0,"wifi tidak sampai kamar, tidak tersedia heater...",wifi tidak sampai kamar tidak tersedia heater ...,wifi tidak sampai kamar tidak tersedia heater ...,wifi tidak sampai kamar tidak tersedia heater ...,wifi sampai kamar tersedia heater kopiteh disp...,wifi sampai kamar sedia heater kopiteh dispens...
2,2,1f4447c73e5e1aac462d05b0ecbd5a55,"Lantai kamar mandi kotor, bau, dan seperti sdh...",0,"Lantai kamar mandi kotor, bau, dan seperti sdh...",Lantai kamar mandi kotor bau dan seperti sdh j...,lantai kamar mandi kotor bau dan seperti sdh j...,lantai kamar mandi kotor bau dan seperti sdh j...,lantai kamar mandi kotor bau seperti sdh jdi s...,lantai kamar mandi kotor bau seperti sdh jdi s...
3,3,b9aefbfb0debbfff43f8b268c82b43c0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",0,"Wc jorok.. Kasur tidak dibersihkan,, handuk ti...",Wc jorok Kasur tidak dibersihkan handuk tidak ...,wc jorok kasur tidak dibersihkan handuk tidak ...,wc jorok kasur tidak dibersihkan handuk tidak ...,wc jorok kasur dibersihkan handuk diganti,wc jorok kasur bersih handuk ganti
4,4,e2afef18bd695fae43d4d53b8c547bd4,bocor lagi,0,bocor lagi,bocor lagi,bocor lagi,bocor lagi,bocor,bocor


In [33]:
df3 = df2[['stemmed', 'category']].dropna()
X_train, X_test, y_train, y_test = train_test_split(df3['stemmed'], df3.category, test_size=0.33, random_state=42, stratify=df3.category)

pipeline_tf.fit(X_train, y_train)
pipeline_tf.score(X_test,y_test)


0.9418129848917925

In [34]:
df_test2 = pd.read_csv("../input/haikaldata/cleaned_test_v2.csv")



df_test2['category'] = pipeline_tf.predict(df_test2['stemmed'])

In [35]:
df_test2[['review_id', 'category']].to_csv("submission_v2.csv", index=False)